In [ ]:
"""
En este paso filtramos el dataset de alumnos por el año 2016 y hacemos join con el dataset de escuelas.
Luego eliminamos NULLs y grabamos el file df_final.csv. Pensando en predecir el campo nivel_desemp_matematica, 
lo transofrmamos en dos categorias, SATIFACTORIO con valor 0 (satifactorio y avanzado) y 1 BÁSICO (básico y por debajo de básico).
Para ejecutar este paso deben tener en la carpeta /desafio3/datasets , los datasets app_alumno.csv y app_escuela.csv.
"""

import pandas as pd
df = pd.read_csv('../Desafio 3/datasets/app_alumno.csv', parse_dates=True, na_values=['nc'], low_memory=False)
df_escuelas = pd.read_csv('../Desafio 3/datasets/app_escuela.csv', parse_dates=True, na_values=['nc'], low_memory=False)
df_provincia = pd.read_csv('../Desafio 3/datasets/app_provincia.csv', parse_dates=True, na_values=['nc'], low_memory=False)
df_departamento = pd.read_csv('../Desafio 3/datasets/app_departamento.csv', parse_dates=True, na_values=['nc'], low_memory=False)
df_salario = pd.read_csv('../Desafio 3/datasets/app_salario.csv', parse_dates=True, na_values=['nc'], low_memory=False)
df_gastonivel = pd.read_csv('../Desafio 3/datasets/app_gastonivel.csv', parse_dates=True, na_values=['nc'], low_memory=False)
df_gastoobjeto = pd.read_csv('../Desafio 3/datasets/app_gastoobjeto.csv', parse_dates=True, na_values=['nc'], low_memory=False)
df_poblacion = pd.read_excel('../Desafio 3/datasets/poblacion por provincia.xlsx')

In [2]:
# Limpieza del dataset "original"
df = df[(df['nivel_desemp_matematica'].notnull()) & (df['year_id'] == 2016)]
df = df.drop(['id','repeticion_secundaria','nivel_desemp_ciencias_sociales','nivel_desemp_ciencias_naturales','year_id'], axis=1)
df = df.dropna(axis=0)
df['nivel_desemp_matematica'] = df['nivel_desemp_matematica'].apply(lambda x: 0 if x in ['satisfactorio','avanzado'] else 1)

In [3]:
# Limpieza del dataset de escuelas
df_escuelas = df_escuelas.rename(columns={'id': 'escuela_id'})
df_escuelas = df_escuelas.drop('icse_cat', axis=1)


In [4]:
# Limpieza del dataset de salario
df_salario = df_salario[(df_salario['year_id'] == 2016)]
df_salario = df_salario.groupby(['provincia_id', 'name'])['value'].aggregate('mean').unstack()


In [5]:
df_salario

name,director_primaria,director_secundaria,maestro_de_grado,maestro_inicial,profesor_secundaria_15hs,profesor_snu
provincia_id,,,,,,
0,18795.31,16680.16,12033.14,12033.14,9306.40,9306.40
1,13807.37,15052.58,9137.21,9210.75,9129.61,9126.91
2,14730.69,17607.31,12036.97,12417.97,10676.76,10265.28
3,29250.37,28743.28,14753.74,14753.74,11065.31,10411.72
4,27996.25,27099.79,11744.66,11744.66,8995.75,7937.70
5,20488.88,20488.88,10385.56,10469.81,9463.53,9410.44
6,21111.32,24021.32,16149.26,16149.26,11188.21,10359.99
7,17510.35,18653.24,11715.67,11820.31,7871.23,8116.96
8,16637.84,23651.81,9715.07,9881.27,9731.69,9721.71


In [6]:
# Limpieza del dataset de gasto nivel
df_gastonivel = df_gastonivel[(df_gastonivel['year_id'] == 2016)]
df_gastonivel = df_gastonivel.groupby(['provincia_id', 'name'])['value'].aggregate('mean').unstack()

df_gastonivel = df_gastonivel.merge(df_poblacion, how='inner', left_on=['provincia_id'],right_on=['id'])

df_gastonivel = df_gastonivel[['inicial+primaria', 'privada', 'secundaria', 'sin_discriminar', 'snu']].div(df_gastonivel.poblacion, axis=0)

df_gastonivel['provincia_id'] = df_gastonivel.index

In [8]:
df_gastonivel

,inicial+primaria,privada,secundaria,sin_discriminar,snu,provincia_id
0,2596.019912,923.127478,2429.997161,1135.624953,326.862711,0
1,5793.271151,1356.583256,4675.693971,1440.394769,1052.284649,1
2,5372.593971,975.447972,4038.223831,1716.850584,681.349810,2
3,7704.392738,679.790382,5198.510839,2337.913066,1076.255796,3
4,3459.381441,1570.304930,2324.249551,1300.041646,626.927842,4
5,3235.103664,887.275129,4000.798767,657.280720,682.927251,5
6,2926.410493,1782.640937,2602.015661,390.757999,437.982289,6
7,4483.908141,1596.288683,3845.966277,348.881359,331.107847,7
8,5724.724145,350.303711,4218.216788,608.667934,910.485584,8
9,5064.893172,607.419795,4268.715547,722.322299,413.034920,9


In [9]:
# Limpieza del dataset de gasto objeto
df_gastoobjeto = df_gastoobjeto[(df_gastoobjeto['year_id'] == 2016)]
df_gastoobjeto = df_gastoobjeto.groupby(['provincia_id', 'name'])['value'].aggregate('mean').unstack()

df_gastoobjeto = df_gastoobjeto.merge(df_poblacion, how='inner', left_on=['provincia_id'],right_on=['id'])

df_gastoobjeto = df_gastoobjeto[['bienes_y_servicios_no_personales', 'erogacion_de_capital', 'personal', 'transferencias_educacion_privada', 'transferencias_otras']].div(df_gastoobjeto.poblacion, axis=0)

df_gastoobjeto['provincia_id'] = df_gastoobjeto.index

In [10]:
df_gastoobjeto

,bienes_y_servicios_no_personales,erogacion_de_capital,personal,transferencias_educacion_privada,transferencias_otras,provincia_id
0,113.910918,91.242470,6278.975893,923.127478,4.375457,0
1,455.511513,539.491981,11664.081012,1356.583255,302.560033,1
2,291.415870,340.427614,10903.656889,975.447972,273.517822,2
3,601.573597,630.187388,14998.498341,679.790383,86.813113,3
4,748.074394,375.691067,6253.044614,1570.304930,333.790405,4
5,335.635958,375.403124,7862.335102,887.275129,2.736219,5
6,237.287767,198.123118,5918.518382,1782.640937,3.237176,6
7,148.961765,342.570633,8451.558552,1596.288683,66.772674,7
8,223.686225,1619.695945,9610.472706,350.303710,8.239575,8
9,19.601074,263.926781,10164.323258,607.419795,21.114826,9


In [11]:
# Merge de los dataset limpios
df = df.merge(df_escuelas, how='inner', on ='escuela_id')
df = df.merge(df_salario, how = 'inner', on = 'provincia_id')
df = df.merge(df_gastonivel, how = 'inner', on = 'provincia_id')
df = df.merge(df_gastoobjeto, how = 'inner', on = 'provincia_id')

In [12]:
df.head(4)

,sexo,indice_socioeconomico,nivel_desemp_matematica,nivel_desemp_lengua,ponderador_lengua,ponderador_matematica,ponderador_ciencias_naturales,ponderador_ciencias_sociales,tiene_notebook,tiene_pc,...,inicial+primaria,privada,secundaria,sin_discriminar,snu,bienes_y_servicios_no_personales,erogacion_de_capital,personal,transferencias_educacion_privada,transferencias_otras
0,f,medio,0,avanzado,1.309467,1.308268,-1.0,-1.0,no,si,...,3459.381441,1570.30493,2324.249551,1300.041646,626.927842,748.074394,375.691067,6253.044614,1570.30493,333.790405
1,f,alto,0,avanzado,1.309467,1.308268,-1.0,-1.0,si,si,...,3459.381441,1570.30493,2324.249551,1300.041646,626.927842,748.074394,375.691067,6253.044614,1570.30493,333.790405
2,m,medio,0,satisfactorio,1.309467,1.308268,-1.0,-1.0,si,si,...,3459.381441,1570.30493,2324.249551,1300.041646,626.927842,748.074394,375.691067,6253.044614,1570.30493,333.790405
3,m,medio,0,avanzado,1.309467,1.308268,-1.0,-1.0,no,si,...,3459.381441,1570.30493,2324.249551,1300.041646,626.927842,748.074394,375.691067,6253.044614,1570.30493,333.790405


In [14]:
df.shape

(311428, 47)

In [15]:
# Veo la cantidad de almunos por provincia
cantidad = pd.DataFrame(df.provincia_id.value_counts())
cantidad['id'] = cantidad.index
cantidad = cantidad.merge(df_provincia, how = 'inner', on = 'id')
cantidad

,provincia_id,id,name
0,113692,0,Buenos Aires
1,35463,6,Córdoba
2,20871,20,Santa Fe
3,16046,12,Mendoza
4,12377,23,Tucumán
5,12341,16,Salta
6,10500,13,Misiones
7,10478,7,Entre Ríos
8,9190,4,Ciudad de Buenos Aires
9,8583,5,Corrientes


In [16]:
df_escuelas.describe()

,escuela_id,icse,indice_socioeconomico_medio,departamento_id,provincia_id
count,6.573200e+04,65732.000000,65732.000000,48786.000000,65732.000000
mean,7.789938e+09,-0.408552,0.210513,109.678063,7.648756
std,8.089074e+09,0.708032,1.424044,76.839833,7.567063
min,6.000030e+07,-1.000000,-1.000000,0.000000,0.000000
25%,1.585786e+09,-1.000000,-1.000000,40.000000,0.000000
50%,3.904090e+09,-1.000000,-1.000000,99.000000,6.000000
75%,1.254251e+10,0.332000,1.783916,176.000000,13.000000
max,4.500890e+10,0.911000,3.000000,250.000000,23.000000


In [17]:
df = df.dropna(axis=0)
print('#######SHAPE:','\n',df.shape,'\n','#######','\n')
print('#######VALUE_COUNTS:','\n',df['nivel_desemp_matematica'].value_counts(),'\n','#######','\n')
print('#######NULLs:','\n',(df.isnull().sum()/df.shape[0])*100,'\n','#######','\n')
print('#######BALANCE:','\n',df['nivel_desemp_matematica'].value_counts(),'\n','#######','\n')
df.to_csv('../Desafio 3/datasets/df_final.csv')

#######SHAPE: 
 (293689, 47) 
 ####### 

#######VALUE_COUNTS: 
 0    176110
1    117579
Name: nivel_desemp_matematica, dtype: int64 
 ####### 

#######NULLs: 
 sexo                                0.0
indice_socioeconomico               0.0
nivel_desemp_matematica             0.0
nivel_desemp_lengua                 0.0
ponderador_lengua                   0.0
ponderador_matematica               0.0
ponderador_ciencias_naturales       0.0
ponderador_ciencias_sociales        0.0
tiene_notebook                      0.0
tiene_pc                            0.0
tiene_tablet                        0.0
tiene_celular                       0.0
tiene_smartphone                    0.0
tiene_consola                       0.0
tiene_smarttv                       0.0
tiene_cable                         0.0
tiene_internet_x                    0.0
repeticion_primaria                 0.0
escuela_id                          0.0
nivel_id                            0.0
ambito                              0.0


In [18]:
"""
En este paso, levantamos el nuevo dataset y convertimos al tipo de dato category las columnas que consideramos categoricas
"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import RobustScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import make_pipeline, make_union, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn import metrics
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

df = pd.read_csv('../Desafio 3/datasets/df_final.csv', parse_dates=True, na_values=['nc'], low_memory=False)
df = df.drop('Unnamed: 0', axis=1)
for i in df.columns:
    if (df[i].dtype == object) or i in ['nivel_id', 'escuela_id', 'provincia_id', 'departamento_id']:
        df[i] = pd.Categorical(df[i])




Bad key "text.kerning_factor" on line 4 in
C:\Users\gbulgach\AppData\Local\Continuum\anaconda3\envs\dhdsblend\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [19]:

"""A continuación ejecutamos modelos de regresión logistica para validar que el dataset final sea adecuado para trabajar .
"""
X = df.drop('nivel_desemp_matematica', axis=1)
y = df['nivel_desemp_matematica']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state=42)


In [ ]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns = 'n'):
        self.columns = columns
    
    def transform(self, X, *_):
        if isinstance(X, pd.DataFrame):
            #c para tomar columnas categoricas, cualquier otro valor para tomar continuas.
            if self.columns == 'c':
                categorical_columns = [col for col in X.columns if pd.api.types.is_categorical_dtype(X[col])]
                return pd.DataFrame(X[categorical_columns])
            else:
                scal_columns = [col for col in X.columns if (pd.api.types.is_categorical_dtype(X[col]) == False)]
                return pd.DataFrame(X[scal_columns])
        else:
            raise TypeError("Este Transformador solo funciona en DF de Pandas")
    
    def fit(self, X, *_):
        return self
cat_pipe = make_pipeline(ColumnSelector('c'),OneHotEncoder(handle_unknown='ignore'))
num_pipe = make_pipeline(ColumnSelector('n'),StandardScaler())
union = make_union(num_pipe,
                   cat_pipe)



In [ ]:
#Este paso tarda bastante dado que ejecutamos un random grid search.
"""
RESULTADO: 
Best score: 0.739
Best parameters set:
	 log__C: 0.1
	 log__penalty: 'l2'
              precision    recall  f1-score   support

           0       0.77      0.81      0.79     44028
           1       0.69      0.63      0.66     29395

    accuracy                           0.74     73423
   macro avg       0.73      0.72      0.72     73423
weighted avg       0.73      0.74      0.74     73423

0.7373302643585797

##########################################################################################

RESULTADO CON MAS FEATURES:
Best score: 0.738
Best parameters set:
	 log__C: 0.1
	 log__penalty: 'l2'
              precision    recall  f1-score   support

           0       0.77      0.80      0.79     44028
           1       0.69      0.64      0.66     29395

    accuracy                           0.74     73423
   macro avg       0.73      0.72      0.73     73423
weighted avg       0.74      0.74      0.74     73423

0.7399316290535664



"""
union.fit_transform(X)

pipeline = Pipeline([('union', union), 
                     ('log', LogisticRegression(solver='liblinear'))])

parameters = {'log__penalty': ['l1', 'l2'],
              'log__C': [0.01, 0.1, 1, 10, 100]}
folds=StratifiedKFold(n_splits=5,shuffle=True, random_state=42)
grid_search = RandomizedSearchCV(pipeline, parameters,cv=folds, n_jobs = 5)
grid_search.fit(pd.DataFrame(X_train, columns=X.columns), y_train)

print("Best score: %0.3f" % grid_search.best_score_) 
print("Best parameters set:" )
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted (parameters . keys()): 
                    print("\t %s: %r" % (param_name, best_parameters[param_name])) 
y_pred = grid_search.best_estimator_.predict(X_test)
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

In [ ]:
print('\nconfusion:\n',confusion_matrix(y_test, y_pred))